# Mercado Libre
Al parecer cada página del listado de ML tiene 51 anuncios (publicaciones) y en cada una hay 3 publicaciones de promocionados (o sea, 3 productos repetidos y 48 normales)

In [77]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup as bs
from multiprocessing import Pool
import inspect
import itertools
from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
#import chromedriver_binary

## Extracción de features de cada publicación
Aquí voy a sacar todos los features de cada publicación en cada página del listado

In [78]:
busqueda = 'bota desechable'
modo_rapido = True

In [79]:
# Saber cuántas publicaciones hay en total
search_keyword = busqueda.replace(' ','-')
url_root = 'https://listado.mercadolibre.com.mx/%s_Desde_'%(search_keyword)
#url_root = 'https://listado.mercadolibre.com.mx/%s_CostoEnvio_Gratis_Desde_'%(search_keyword)
r = requests.get(url_root+'1')
soup = bs(r.text)
cant_resultados = soup.find('span', attrs={'class':'ui-search-search-result__quantity-results'}).get_text()
cant_resultados = pd.to_numeric(''.join(re.findall('\d', cant_resultados)))
cant_resultados

410

In [80]:
#Ir página por página del listado y sacar los features de cada publicación (item)
#driver = webdriver.Chrome(r'C:\Users\Dell-G3\anaconda3\Lib\site-packages\chromedriver_binary\chromedriver.exe') #Aquí está el chromedriver pero se tiene que ir actualizando
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

#driver.maximize_window()

if modo_rapido:
    links_of_pages = [url_root + str(num) for num in np.arange(1,49+1,48)]
else:
    links_of_pages = [url_root + str(num) for num in np.arange(1,cant_resultados+1,48)]

list_of_pages_features = []
page = 0
for link in links_of_pages:
    page += 1
    driver.get(link)
    html = driver.page_source
    soup = bs(html)
    items = soup.find_all('li', attrs={'class':'ui-search-layout__item'})
    page_features = []
    for item in items:
        item_features_dict = {}
        item_features_dict['pagina'] = page
        
        titulo = item.select_one('div h2.ui-search-item__title')
        if titulo is not None : #Me cercioro que sí haya encontrado 1 y solo 1 resultado 
            item_features_dict['titulo'] = titulo.get_text()
                
        precio = item.select_one('span.andes-money-amount__fraction') 
        if precio is not None:
            precio = ''.join(re.findall(r'\d',precio.get_text()))
            item_features_dict['precio'] = pd.to_numeric(precio)
        
        pct_descuento = item.select_one('span.ui-search-price__discount')
        if pct_descuento is not None:
            pct_descuento = ''.join(re.findall('\d',pct_descuento.get_text()))
            item_features_dict['pct_descuento'] = pd.to_numeric(pct_descuento)
        else:
            item_features_dict['pct_descuento'] = 0
            
        is_full = item.select_one('svg.ui-search-icon--full')
        if is_full is not None:
            item_features_dict['is_full'] = True
        else:
            item_features_dict['is_full'] = False
        
        is_envio_gratis = item.find('p', class_='meliplus--actived')
        if is_envio_gratis is not None:
            item_features_dict['is_envio_gratis'] = True
            #item_features_dict['envio_object'] = is_envio_gratis
        elif ((item.find('span', class_='ui-search-item__promise__text',string=re.compile('gratis')))is not None):
            item_features_dict['is_envio_gratis'] = True
        else:
            item_features_dict['is_envio_gratis'] = False
        
        is_promocionado = item.find('label', attrs={'class': 'ui-search-item__highlight-label__text'})
        if is_promocionado is not None:
            item_features_dict['is_promocionado'] = True
        else:
            item_features_dict['is_promocionado'] = False
        
        link_of_item = item.find('a')
        if link_of_item is not None:
            parts_of_link = link_of_item['href'].partition('-_JM#')
            item_features_dict['link_of_item'] = parts_of_link[0]+parts_of_link[1]
        
        page_features.append(item_features_dict)    
    list_of_pages_features.append(page_features)
driver.quit()
list_of_all_features = list(itertools.chain.from_iterable(list_of_pages_features))
primary_features = pd.DataFrame(list_of_all_features)
primary_features 

,pagina,titulo,precio,pct_descuento,is_full,is_envio_gratis,is_promocionado,link_of_item
0,1,Botas Quirúrgicas Desechables 50 Piezas,303,11,False,False,False,https://click1.mercadolibre.com.mx/mclics/clic...
1,1,Bota Quirúrgica Desechable Sin Plantilla Demev...,147,0,True,False,True,https://articulo.mercadolibre.com.mx/MLM-15909...
2,1,"Botas Desechables Neogen Para Mujer, Azules, T...",1425,0,False,True,False,https://articulo.mercadolibre.com.mx/MLM-18195...
3,1,"Bota Quirúrgica Desechable Tela Sms , 50piezas",151,4,False,False,False,https://articulo.mercadolibre.com.mx/MLM-85097...
4,1,"Bota Quirúrgica Desechable Tela Sms , 100piezas",281,4,False,False,False,https://articulo.mercadolibre.com.mx/MLM-13904...
...,...,...,...,...,...,...,...,...
103,2,Bota Desechable Para Cirujano Con Plantilla C/...,610,0,False,True,False,https://articulo.mercadolibre.com.mx/MLM-15904...
104,2,Bota Quirúrgica Desechable (par),35,0,False,False,False,https://articulo.mercadolibre.com.mx/MLM-18873...
105,2,Bota Desechable Para Cirujano Sin Plantilla (2...,187,0,False,False,False,https://articulo.mercadolibre.com.mx/MLM-77013...
106,2,Botas Quirúrgicas Desechables 50 Pares Quirófano,447,4,False,True,False,https://click1.mercadolibre.com.mx/mclics/clic...


In [89]:
# Entrar a cada publicación y extraer features secundarias
def parallal_task(func, iterable):

    with open(f'./tmp_func.py', 'w') as file:
        file.write('import pandas as pd\n'
                   'import numpy as np\n'
                   'import requests\n'
                   'import re\n'
                   'from bs4 import BeautifulSoup as bs\n')
        file.write(inspect.getsource(func).replace(func.__name__, "task"))


    from tmp_func import task

    if __name__ == '__main__':
        pool = Pool(processes=20)
        res = pool.map(task, iterable)#task es en realidad scrape_second... pero la llama desde el .py creado
        pool.close()
        return res #Regresa una lista de dicts al llamado de parallal_task
    else:
        raise "Not in Jupyter Notebook"
        
def scrape_secondary_features(url):
    item_secondary_features_dict = {}
    r = requests.get(url)
    publicacion = bs(r.text)
    
    item_secondary_features_dict['link_of_item'] = url
    
    item_id = publicacion.find('input', attrs={'name':'item_id'}) 
    if item_id is not None:
        item_secondary_features_dict['item_id'] = item_id['value']
    
    cant_ventas = publicacion.find('span', attrs={'class':'ui-pdp-subtitle'} ,string=re.compile('vendidos'))
    if cant_ventas is not None:
        cant_ventas = ''.join(re.findall('\d', cant_ventas.get_text()))
        item_secondary_features_dict['vendidos'] = pd.to_numeric(cant_ventas)
    else:
        item_secondary_features_dict['vendidos'] = 0
    
    is_nuevo = publicacion.find('span', attrs={'class':'ui-pdp-subtitle'} ,string=re.compile('(n|N)uevo'))
    if is_nuevo is not None:
        item_secondary_features_dict['is_nuevo'] = True
    else:
        item_secondary_features_dict['is_nuevo'] = False
    
    nivel_vendedor = publicacion.find('p', attrs={'class':'ui-pdp-seller__status-title'})
    if nivel_vendedor is not None:
        item_secondary_features_dict['nivel_vendedor'] = nivel_vendedor.get_text()
    else:
        r = requests.get(url)
        publicacion = bs(r.text)
        nivel_vendedor = publicacion.find('p', attrs={'class':'ui-pdp-seller__status-title'})
        if nivel_vendedor is not None:
            item_secondary_features_dict['nivel_vendedor'] = nivel_vendedor.get_text()
        else:
            r = requests.get(url)
            publicacion = bs(r.text)
            nivel_vendedor = publicacion.find('p', attrs={'class':'ui-seller-data-status__title'}) 
            if nivel_vendedor is not None:
                item_secondary_features_dict['nivel_vendedor'] = nivel_vendedor.get_text()

    ventas_2_meses = publicacion.find('p', attrs={'class':'ui-seller-data-status__info-title'})
    if ventas_2_meses is not None:
        ventas_2_meses = ''.join(re.findall('\d', ventas_2_meses.get_text()))
        item_secondary_features_dict['ventas_2_meses'] = pd.to_numeric(ventas_2_meses)
        
    return (item_secondary_features_dict)  #Regresa un dict al llamado de pool.map

links = primary_features.link_of_item
all_items_secondary_features = parallal_task(scrape_secondary_features, links)
secondary_features = pd.DataFrame(all_items_secondary_features)
features_raw = primary_features.merge(secondary_features, on='link_of_item')
features_raw_sorted = features_raw.sort_values('is_promocionado', ascending=False)
features = features_raw_sorted.drop_duplicates(subset='item_id').reset_index(drop=True)

In [90]:
#Feature engineering: augmenting the DF
envios_forzosos = [True if ((row['is_envio_gratis'] == True) & (row['precio']>= 299)) else False for index,row in features.iterrows()]
features.loc[:,'is_envio_forzoso'] = envios_forzosos

precios_anteriores = features['precio'].divide((100-features.pct_descuento)/100, axis='rows')
features.loc[:,'precio_anterior'] = precios_anteriores

features = features[['item_id', 'titulo', 'precio', 'vendidos', 'nivel_vendedor', 'ventas_2_meses',  'pct_descuento','precio_anterior' ,'is_full',
                       'is_envio_gratis','is_envio_forzoso','is_promocionado', 'is_nuevo', 'link_of_item', 'pagina']]

## Guardar los DF diarios para monitoreo de variables

In [91]:
from datetime import date
from pathlib import Path

today=str(date.today())
output_file = (today+'-'+search_keyword+'.csv') 
output_dir = Path(r'C:\Users\Dell-G3\Documents\Jupyter-projects\Webscrapping\monitoreo_diario\%s'%(search_keyword))
output_dir.mkdir(parents=True, exist_ok=True)
features.to_csv(output_dir / output_file, index = False, encoding='utf-8-sig')  # can join path elements with / operator

In [8]:
#features.sort_values('vendidos', ascending=False).head(10)                                                      

## Sandbox
Hago todo esto para saber cuantos anuncios en total hay en cada página de la búsqueda y también cuántos de esos son promocionados

In [9]:
"""
itera=features[features.nivel_vendedor.isna()].link_of_item.values
len(itera)
"""

'\nitera=features[features.nivel_vendedor.isna()].link_of_item.values\nlen(itera)\n'

In [10]:
"""
for link in itera:
    item_secondary_features_dict = {}
    r = requests.get(link)
    publicacion = bs(r.text)

    nivel_vendedor = publicacion.find('h3', attrs={'class':'ui-pdp-seller__status-title'})
    if nivel_vendedor is not None:
        item_secondary_features_dict['nivel_vendedor'] = nivel_vendedor.get_text()
    a = pd.DataFrame([item_secondary_features_dict])
    print(a)
    """
    

"\nfor link in itera:\n    item_secondary_features_dict = {}\n    r = requests.get(link)\n    publicacion = bs(r.text, 'html5lib')\n\n    nivel_vendedor = publicacion.find('h3', attrs={'class':'ui-pdp-seller__status-title'})\n    if nivel_vendedor is not None:\n        item_secondary_features_dict['nivel_vendedor'] = nivel_vendedor.get_text()\n    a = pd.DataFrame([item_secondary_features_dict])\n    print(a)\n    "

In [11]:
#a=primary_features.drop_duplicates(subset=['titulo','precio','is_full','is_envio_gratis'])    
#a.is_envio_gratis.value_counts().values.sum()

#b=a.reindex(primary_features.index)
#b.loc[b.isna().all(axis=1)].shape
#a=primary_features[primary_features.duplicated(subset=['titulo','precio','is_full','is_envio_gratis'])]    
#a=primary_features.loc[~primary_features.link_of_item.apply(lambda x: 'click1' in x)]
#a.is_envio_gratis.value_counts()

In [12]:
# TODO:   
#         1. Sacar si la publicación está pagando premium o no por los meses sin interés
#         2. Ver cuántas estrellas y # de opiniones
#         3. Ver cuántos disponibles tiene
#         4. Ver categorias
#         5. Analizar características de productos en primera página
#         5. Sacar estadísticas de precios y ver relación con demás variables
#         6. Usar visualizaciones matplotlib para resultados
#         7. Sacar una distribución de cómo ML ordena las publicaciones premium, clásica, promocionada.
#         8. Extrapolar esas ideas y ver cómo está la distribución de otras features con respecto al número de página
#         9. También se puede comparar eso con el número de ventas de una publicación y su posición en los listados en varios días
#        10. Reverse Engineering al algoritmo de ML
#        12. Scrapear automáticamente todos los productos de una lista con nuestros precios de compra y en 
#            automático que reconozca qué productos tienen un precio promedio en donde podamos competir
#        14. Eficientar todo el código separando funciones en scripts y usando multiprocessing, multithreading y fast loops.